### Single-input Network - PyTorch

- split jupyter notebooks (dataset creation, network training)

###### waypoint executer
- write program that finds path between waypoints

###### training
- make use of tensorboard
- load and save models

###### testing (most important)
- Classification network to predict coordinates
- Multiple action prediction (How to shuffle data for long seq.?) /  create dataset with n steps appart
- Siam network: https://colab.research.google.com/drive/1sn7BDKVvi8-Ng37gvfyNw8OCf8kZY91o?usp=sharing

In [ ]:
'''
Imports external and own libraries
'''

import pickle

import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader

from prettytable import PrettyTable

# own
import collector
import action
import world
import plot
import preprocess
import nets
import train

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
'''
Use Dataloader to make the data ready for the training loop
'''
train_data, test_data = preprocess.split_data(oracle_random_data, 0.8)

# preprocess trainingset 
oracle_train_data = preprocess.ActionObservationDataset(train_data)
oracle_test_data = preprocess.ActionObservationDataset(test_data)

# build dataloader (tensor format)
batch_size = 64
dataset_loader_train_data = DataLoader(oracle_train_data, batch_size=batch_size, shuffle=True)
dataset_loader_test_data = DataLoader(oracle_test_data, batch_size=batch_size, shuffle=True)

In [ ]:
'''
Visualize one batch
'''

# next data part
dataiter = iter(dataset_loader)
# plot 64 examples
images, labels = dataiter.next()
plot.plot_64_observations(images)

In [ ]:
'''
Feed-forward network
'''

forward = nets.Forward()
summary(forward, (3,32,32))

forward = nets.Forward_Large()
summary(forward, (3,32,32))

In [ ]:
'''
Train the network
'''
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(forward.parameters(), lr=0.3, momentum=0.9) # for small one 0.01 works well

episodes = 10
net, test_loss, train_loss, test_acc, train_acc  = train.train(dataset_loader_train_data, dataset_loader_test_data, forward, criterion, optimizer, episodes)


In [ ]:
'''
Plot loss and accuracy curves for training and test set
'''

plot.plot_losses(test_loss, train_loss)
plot.plot_acc(test_acc, train_acc, smooth=True)

In [ ]:
import torch

def show_example_classificataions(dataset, net, amount=5):
    classes_expl = {0: 'turn left', 1: 'turn right', 2: 'walk forwards', 3: 'walk backwards'} 
    
    # calculate total accuracy on training data
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in dataset:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('GroundTruth: ', ', '.join(f'{classes_expl[int(labels[j])]}' for j in range(amount)))
    print('Predicted: ', ', '.join(f'{classes_expl[int(predicted[j])]}' for j in range(amount)), '\n')

import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(dataset, network, save=False):
    classes_expl = {0: 'turn left', 1: 'turn right', 2: 'walk forwards', 3: 'walk backwards'} 
    
    # plot confusion matrix
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in dataset_loader_train_data:
            output = net(inputs) # Feed pass through network

            output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
            y_pred.extend(output) # Save Prediction

            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth

    # Build confusion matrix
    classes = list(set(labels))
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [classes_expl[i] for i in classes],
                         columns = [classes_expl[i] for i in classes])
    plt.figure(figsize = (12,7))
    plt.rcParams.update({'font.size': 16})
    sn.heatmap(df_cm, annot=True)
    plt.suptitle('Confusion matrix')
    plt.ylabel('True labels')
    plt.xlabel('Predicted labels')
    plt.show()

    if save:
        plt.savefig('output.png')

show_example_classificataions(dataset_loader_train_data, net, amount=8)
plot_confusion_matrix(dataset_loader_train_data, net)

### Multi-input Network - Pytorch

In [ ]:
# https://github.com/zaidalyafeai/Machine-Learning/blob/master/Multi-input%20Network%20Pytorch.ipynb